In [33]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import pytz
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from concurrent.futures import ThreadPoolExecutor, as_completed
import itertools
import logging
import time

# Configures logging for progress and error capture
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
class MarketEnvironment:
    def __init__(self):
        self.us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())
        self.closest_time_index = None

    def adjust_to_trading_hours(self, dt):
        # Ensures datetime falls is in standard trading hours; sets time zone
        if dt.tzinfo is None or dt.tzinfo.utcoffset(dt) is None:
            dt = pytz.timezone('America/New_York').localize(dt)
        if dt.weekday() >= 5:  
            dt += self.us_bd
        if dt.hour < 9 or (dt.hour == 9 and dt.minute < 30):
            dt = dt.replace(hour=9, minute=30)
        elif dt.hour > 16:
            dt = (dt + self.us_bd).replace(hour=9, minute=30)

        return dt

    def fetch_market_data(self, ticker, signal_date):
        # Retrieves minute-level market data around the signal date
        signal_date = self.adjust_to_trading_hours(signal_date.tz_localize(pytz.timezone('America/New_York')))
        start_date_utc = (signal_date - timedelta(days=1)).astimezone(pytz.utc)
        end_date_utc = (signal_date + timedelta(days=6)).astimezone(pytz.utc)
        attempts, wait = 20, 0.001
        for attempt in range(attempts):
            try:
                data = yf.download(ticker, start=start_date_utc.strftime('%Y-%m-%d'), end=end_date_utc.strftime('%Y-%m-%d'), interval='1m', progress=False)
                if data.empty:
                    continue
                data.index = data.index.tz_convert(pytz.timezone('America/New_York'))

                self.closest_time_index = data.index.get_loc(signal_date, method='nearest')
                callout_price = data.at[data.index[self.closest_time_index], 'Close']
                
                # Split the data
                data_for_atr = data.loc[:data.index[self.closest_time_index + 1]]
                data_for_backtest = data.loc[data.index[self.closest_time_index]:]
                
                return data_for_atr, data_for_backtest, callout_price
            except Exception as e:
                time.sleep(wait)
                wait *= 2  # Exponential backoff
        logging.error(f"Failed to download data for {ticker} after {attempts} attempts.")
        return None, None, None

    def calculate_atr(self, data, period):
        # Computes the Average True Range (ATR), a measure of market volatility
        high_low = data['High'] - data['Low']
        high_close = np.abs(data['High'] - data['Close'].shift())
        low_close = np.abs(data['Low'] - data['Close'].shift())
        ranges = pd.concat([high_low, high_close, low_close], axis=1)
        return np.max(ranges, axis=1).rolling(window=period).mean()


class Backtester:
    def __init__(self, initial_capital=10000):
        # Sets up the backtester with an initial capital and market environment
        self.initial_capital = initial_capital
        self.market_env = MarketEnvironment()

    def run_backtest(self, data_for_atr, data_for_backtest, callout_price, atr_multiplier, trailing_stop_multiplier, atr_period):
        # Executes backtest using atr range and multiplier (both being optimized)
        atr = self.market_env.calculate_atr(data_for_atr, atr_period).iloc[-1] * atr_multiplier
        profit_losses = ((data_for_backtest['Close'] - callout_price) / callout_price * 100).tolist()
        return self.evaluate_trades(profit_losses, atr, trailing_stop_multiplier)

    def evaluate_trades(self, profit_losses, atr, trailing_stop_multiplier):
        # Evaluates trading signals and manages trades within a simulated portfolio
        portfolio = {
            'Capital': self.initial_capital,
            'Cash': self.initial_capital,
            'Equity': 0,
            'Returns': [],
            'Drawdowns': [],
            'Successful Trades': 0
        }

        initial_investment = portfolio['Cash']
        portfolio['Cash'] -= initial_investment
        portfolio['Equity'] = initial_investment
        max_profit_loss = 0
        minutes_taken = 0

        for i, profit_loss in enumerate(profit_losses):
            if profit_loss > max_profit_loss:
                max_profit_loss = profit_loss

            if profit_loss < max_profit_loss - (atr * trailing_stop_multiplier) or i == len(profit_losses) - 1:
                sell_amount = portfolio['Equity']
                drawdown = ((1 + max_profit_loss / 100) / (1 + profit_loss / 100)) - 1
                portfolio['Drawdowns'].append(drawdown)
                portfolio['Cash'] += sell_amount * (1 + profit_loss / 100)
                portfolio['Equity'] -= sell_amount
                portfolio['Returns'].append(profit_loss / 100)
                minutes_taken = i
                if profit_loss > 1 and drawdown < 0.005:
                    portfolio['Successful Trades'] += 1
                break

        total_return = (portfolio['Cash'] - self.initial_capital) / self.initial_capital
        portfolio_variance = np.var(portfolio['Returns']) if portfolio['Returns'] else 0
        sharpe_ratio = total_return / np.sqrt(portfolio_variance) if portfolio_variance else 0
        max_drawdown = max(portfolio['Drawdowns']) if portfolio['Drawdowns'] else 0
        avg_trade_gain = np.mean(portfolio['Returns']) if portfolio['Returns'] else 0

        return {
            'Total Return': total_return,
            'Portfolio Variance': portfolio_variance,
            'Sharpe Ratio': sharpe_ratio,
            'Final Equity': portfolio['Cash'],
            'Maximum Drawdown': max_drawdown,
            'Average Trade Gain': avg_trade_gain,
            'Successful Trades': portfolio['Successful Trades'],
            'Minutes Taken': minutes_taken
        }

def optimize_strategy(ticker, created_at, data_for_atr, data_for_backtest, callout_price, param_ranges, backtester):
    # Initialize an empty list to store results
    results = []
    
    # Iterate over all combinations of atr_multiplier, trailing_stop_loss_multiplier, and atr_periods
    for atr_mult, stop_mult, atr_period in itertools.product(param_ranges['atr_multiplier'], param_ranges['trailing_stop_loss_multiplier'], param_ranges['atr_periods']):
        
        # Run the backtest with the current combination of parameters
        result = backtester.run_backtest(data_for_atr, data_for_backtest, callout_price, atr_mult, stop_mult, atr_period)
        
        # If the backtest produced a result, append the details to the results list
        if result:
            results.append({
                'Ticker': ticker,
                'created_at': created_at,
                'ATR Multiplier': atr_mult,
                'Trailing Stop Multiplier': stop_mult,
                'ATR Period': atr_period,
                'Total Return': result['Total Return'],
                'Portfolio Variance': result['Portfolio Variance'],
                'Sharpe Ratio': result['Sharpe Ratio'],
                'Final Equity': result['Final Equity'],
                'Maximum Drawdown': result['Maximum Drawdown'],
                'Successful Trades': result['Successful Trades'],
                'Minutes Taken': result['Minutes Taken'],
                # Calculate a score based on Total Return, Maximum Drawdown, and Minutes Taken
                'Score': (result['Total Return'] - result['Maximum Drawdown']) / (result['Minutes Taken'] + 0.0001) * 6000
            })
    
    # Return the list of results for all parameter combinations
    return results

def parallel_optimize_strategy(df, param_ranges):
    # Initialize the backtester
    backtester = Backtester()
    results = []

    def process_row(row):
        # Fetch market data for the specific row (i.e., specific callout)
        data_for_atr, data_for_backtest, callout_price = backtester.market_env.fetch_market_data(row['Ticker'], pd.to_datetime(row['created_at']))
        
        # If data was successfully fetched, run the optimization strategy
        if data_for_atr is not None and data_for_backtest is not None:
            return optimize_strategy(row['Ticker'], row['created_at'], data_for_atr, data_for_backtest, callout_price, param_ranges, backtester)
        else:
            return []

    # Use ThreadPoolExecutor to parallelize the process_row function for each row in the dataframe
    with ThreadPoolExecutor(max_workers=2) as executor:
        futures = {executor.submit(process_row, row): row for _, row in df.iterrows()}
        for future in as_completed(futures):
            result = future.result()
            if result:
                results.extend(result)
    
    # Convert the results list into a DataFrame and return
    return pd.DataFrame(results)


def return_stock(message):
    # Extracts stock symbol from social media messages
    if len(message.split()) > 1:
        return message.split()[0].replace('[','').replace(']','').replace('$', '').replace('\\', '').replace('*','').upper()

def return_buy_sell(message):
    # Determines buy/sell signal from parsed messages
    if len(message.split()) > 1:
        return 1 if 'Buy' in message.split()[1].replace('[','').replace(']','') else 0

if __name__ == "__main__":
    df = pd.read_excel(r"C:\Users\amoog\Desktop\Project_X\Project_X\tweet_buy_or_sell.xlsx")
    df['Ticker'] = df['buy_or_sell'].apply(return_stock)
    df['Buy'] = df['buy_or_sell'].apply(return_buy_sell)
    df = df[(df['Ticker'].notnull()) & (df['Buy'] == 1)]
    param_ranges = {
        'atr_multiplier': np.arange(1, 3.5, 0.5),
        'trailing_stop_loss_multiplier': np.arange(1, 3.5, 0.5),
        'atr_periods': [14, 50, 100, 200, 400, 650]
    }
    results = parallel_optimize_strategy(df, param_ranges)
    best_results = results.loc[results.groupby(['Ticker', 'created_at'])['Total Return'].idxmax()]
    sorted_results = best_results.sort_values(by='Final Equity', ascending=False)
    display(sorted_results)



1 Failed download:
- VVIX: No data found, symbol may be delisted

1 Failed download:
- VVIX: No data found, symbol may be delisted

1 Failed download:
1 Failed download:
- VVIX: No data found, symbol may be delisted

- VVIX: No data found, symbol may be delisted

1 Failed download:
- VVIX: No data found, symbol may be delisted

1 Failed download:
- VVIX: No data found, symbol may be delisted

1 Failed download:
- VVIX: No data found, symbol may be delisted

1 Failed download:
- VVIX: No data found, symbol may be delisted

1 Failed download:
- VVIX: No data found, symbol may be delisted

1 Failed download:
1 Failed download:
- VVIX: No data found, symbol may be delisted



1 Failed download:
1 Failed download:
- VVIX: No data found, symbol may be delisted



1 Failed download:
1 Failed download:
- VVIX: No data found, symbol may be delisted

- VVIX: No data found, symbol may be delisted

1 Failed download:
1 Failed download:
- VVIX: No data found, symbol may be delisted



1 Failed dow

2024-05-25 15:02:09,402 - ERROR - Failed to download data for VVIX after 20 attempts.



1 Failed download:
- VVIX: No data found, symbol may be delisted

- VVIX: No data found, symbol may be delisted

1 Failed download:
- VVIX: No data found, symbol may be delisted

1 Failed download:
- VVIX: No data found, symbol may be delisted

1 Failed download:
- VVIX: No data found, symbol may be delisted

1 Failed download:
- VVIX: No data found, symbol may be delisted


,Ticker,created_at,ATR Multiplier,Trailing Stop Multiplier,ATR Period,Total Return,Portfolio Variance,Sharpe Ratio,Final Equity,Maximum Drawdown,Successful Trades,Minutes Taken,Score
605,BITX,2024-05-09 13:41:16.000000,1.0,1.0,650,0.193920,0.0,0,11939.203762,0.000000,1,1298,0.896396
2254,UNG,2024-05-06 16:55:41.000000,1.0,1.0,400,0.185047,0.0,0,11850.468138,0.000000,1,1544,0.719094
755,HOOD,2024-05-09 12:46:24.000000,1.0,1.0,650,0.181792,0.0,0,11817.920351,0.000000,1,1364,0.799672
1055,BITX,2024-05-09 12:42:17.000000,1.0,1.0,650,0.177870,0.0,0,11778.697263,0.000000,1,1356,0.787034
2854,HOOD,2024-05-06 13:37:26.000001,1.0,1.0,400,0.148055,0.0,0,11480.548834,0.000000,1,1702,0.521933
3598,AMD,2024-05-02 15:42:35.000000,3.0,3.0,400,0.139530,0.0,0,11395.300661,0.000000,1,1171,0.714928
2601,VKTX,2024-05-06 13:51:53.000000,1.5,2.5,200,0.051435,0.0,0,10514.354447,0.007911,0,136,1.920216
2100,SMMT,2024-05-06 16:10:40.000000,1.0,1.0,14,0.040269,0.0,0,10402.685265,0.003591,1,3,73.351902
564,ULTA,2024-05-09 13:41:58.000001,2.5,3.0,14,0.032165,0.0,0,10321.647624,0.010362,0,473,0.276562
1505,INTC,2024-05-08 13:44:20.000000,1.0,1.0,650,0.031738,0.0,0,10317.380254,0.013184,0,1306,0.085242
